---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

# df = pd.Series(doc)
# df.head(10)

In [49]:
def date_sorter():
    
    # Your code here
    df = pd.DataFrame(doc, columns=['text'])
    
    df1 = df['text'].str.extractall(r'(\d{1,2})[\/|\-](\d{1,2})[\/|\-](\d{4}|\d{2})(?=\D)')
    df1.index = df1.index.droplevel(1)
    df1.rename(columns={0:'month', 1:'day', 2:'year'}, inplace=True)
    
    df_post1 = df.drop(df1.index.get_level_values(0).tolist())
    
    df2 = df_post1['text'].str.extractall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?: |-|\.|,)+(\d{1,2})(?: |-|\.|,)+(\d{4}|\d{2})')
    df2.index = df2.index.droplevel(1)
    df2.rename(columns={0:'month', 1:'day', 2:'year'}, inplace=True)
    
    df_post2 = df_post1.drop(df2.index.get_level_values(0).tolist())
    
    df3 = df_post2['text'].str.extractall(r'(\d{1,2})(?: |-|\.|,)+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?: |-|\.|,)+(\d{4}|\d{2})')
    df3.index = df3.index.droplevel(1)
    df3.rename(columns={0:'day', 1:'month', 2:'year'}, inplace=True)
    
    df_post3 = df_post2.drop(df3.index.get_level_values(0).tolist())
    
    df4 = df_post3['text'].str.extractall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?: |-|\.|,)+(\d{4})')
    df4.index = df4.index.droplevel(1)
    df4.rename(columns={0:'month', 1:'year'}, inplace=True)
    df4['day'] = 1
    
    df_post4 = df_post3.drop(df4.index.get_level_values(0).tolist())
    
    df5 = df_post4['text'].str.extractall(r'(\d{1,2})\/(\d{4}|\d{2})')
    df5.index = df5.index.droplevel(1)
    df5.rename(columns={0:'month', 1:'year'}, inplace=True)
    df5['day'] = 1
    
    df_post5 = df_post4.drop(df5.index.get_level_values(0).tolist())
    
    df6 = df_post5['text'].str.extractall(r'(\d{4})')
    df6.index = df6.index.droplevel(1)
    df6.rename(columns={0:'year'}, inplace=True)
    df6['month'] = 1
    df6['day'] = 1
    
    df_final = pd.concat([df1, df2, df3, df4, df5, df6])
    df_final['year'] = df_final['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    
    dic = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12,
           'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 
           'September':9, 'October':10, 'November':11, 'December':12}

    df_final['month'] = df_final['month'].apply(lambda x: dic[x] if x in dic else x)
    
    df_final = df_final.astype({'year':int, 'month':int, 'day':int})
    df_final = df_final.sort_values(by=['year', 'month', 'day'], kind='mergesort') 
    df_final = df_final.reset_index()
    
    return df_final['index'] # Your answer here